<a href="https://colab.research.google.com/github/mitsu-h/BirdCLEF/blob/load_npy/make_spec_npy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import librosa
import librosa.display
import joblib

from tqdm import tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_dir="/content/drive/MyDrive/colab/BirdCLEF/"
data_dir= os.path.join(root_dir, "inputs/")
model_dir = os.path.join(root_dir, "models/")

In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

In [ ]:
!kaggle competitions download -c birdclef-2022

In [ ]:
!unzip birdclef-2022.zip

In [ ]:
train_df = pd.read_csv(os.path.join('/content', 'train_metadata.csv'))
train_df.head()

# データ加工

指定秒数で音声を切り取るジェネレータ

In [ ]:
def cut_audio(sig, sr, cut_sec=5):
    cut_len = sr * cut_sec
    for i in range(0, len(sig), cut_len):
      if i+cut_len <= len(sig):
        yield sig[i:i+cut_len]
      else:
        s = sig[i:-1]
        yield np.pad(s, [0, cut_len-len(s)], 'constant')

5秒毎にカットした音声をメルスペクトログラムに変換

In [ ]:
# Convert augmented audio to Mel Spectrogram
def mel_spec(file_path, n_mels=128, n_fft=None, hop_len=None):
    sig, sr = librosa.load(file_path, sr=None)
    top_db = 80
    n_fft = n_fft or sr // 10
    cut_len = sr // 5
    
    spec = np.stack([librosa.feature.melspectrogram(s, sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels) for s in cut_audio(sig, sr)])
    
    # Convert to decibels
    spec = librosa.power_to_db(spec,ref=np.max)

    return spec.astype(np.float32)

1データの切り取り、可視化

In [ ]:
filename = train_df.loc[0,'filename']
file_path = os.path.join('/content', 'train_audio', filename)
spec = mel_spec(file_path)
spec.shape

In [ ]:
librosa.display.specshow(spec[0])
plt.colorbar()
plt.show()
librosa.display.specshow(spec[1])
plt.show()
librosa.display.specshow(spec[2])
plt.colorbar()

In [ ]:
import IPython.display
IPython.display.Audio(file_path)

# メルスペクトログラムの保存

In [ ]:
def save_spec_npy(filename, save_dir='audio_images'):
  file_path = os.path.join('/content', 'train_audio', filename)
  spec = mel_spec(file_path)

  save_path = os.path.join(data_dir, 'audio_images', filename+'.npy')
  os.makedirs(os.path.dirname(save_path), exist_ok=True)
  np.save(save_path, spec)

joblibで並列処理

In [ ]:
pool = joblib.Parallel(4)
mapper = joblib.delayed(save_spec_npy)
tasks = [mapper(filename) for filename in train_df['filename']]
pool(tqdm(tasks))

In [ ]:
import glob
audio_images = glob.glob(os.path.join(data_dir, 'audio_images')+'/*/*.npy')
len(audio_images)

In [ ]:
audio_images